<center><a href="https://www.nvidia.cn/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>

<br>

# <font color="#76b900">**Notebook 2：** LLM 服务和 AI 基础模型</font>

<br>

在这个 notebook 中，我们将探索 LLM 服务！我们会讨论在边缘设备上和在可扩展服务器上（例如通过 NVIDIA AI Foundation Endpoints）部署 LLM 的优劣势。

<br>

### **学习目标：**

* 了解 LLM 服务在本地运行与在可扩展的云端环境中运行的优劣势。
* 熟悉 AI Foundation Model Endpoint，包括：
	+ 由 `curl` 和 `requests` 支持的底层通信接口。
	+ 为使此接口能无缝的与 LangChain 这类开源软件对接而创建的抽象。
* 熟悉从入口池（endpoints pool）中检索适合您应用的 LLM。

<br> 

### **思考问题：**

1. 您应该为 LLM 应用开发者提供何种模型访问权限？这跟使用 AI Web 应用的终端用户相比有什么不同？
2. 在考虑要支持哪些设备时，您对设备的本地计算资源做出了哪些假设，应做出哪些取舍？
	* 如果您想为客户提供可访问私有 LLM 部署的 Jupyter Labs 界面，该怎么办？
	* 如果您想在本地 Jupyter 实验室环境私有化部署 LLM，该怎么办？
	* 想支持嵌入式设备（比如 Jetson Nano）的话，又该如何？
3. **[更难的问题]**假设您已在共享 GPU 的云端计算实例上部署了 Stable Diffusion、Mixtral 和 Llama-13B。目前暂时没有用到 Stable Diffusion 的业务，但您的团队正在为 LLM 应用试验另两个模型。那么是否应该删除已部署的 Stable Diffusion？

<br>

### **Notebook 版权声明：**

* 本 notebook 是 [**NVIDIA 深度学习培训中心**](https://www.nvidia.cn/training/)的课程[**《构建大语言模型 RAG 智能体》**](https://www.nvidia.cn/training/instructor-led-workshops/building-rag-agents-with-llms/)中的一部分，未经 NVIDIA 授权不得分发。

----

<br>

## **第 1 部分：**: 将大型模型引入环境

回顾上一个 notebook，我们当前的环境在云实例上运行着多个微服务：`docker-router`，`jupyter-notebook-server`，`frontend` 以及 `llm-service` 等。

* **jupyter-notebook-server**：运行此 Jupyter Labs 会话并托管 Python 环境的服务。
* **docker_router**：帮助我们观察和监控微服务的服务。
* **frontend**：为我们提供简单聊天界面的网页微服务。

此 notebook 主要关注 `llm-service` 微服务，您将使用该服务（在后台使用）与几个[**基础模型**](https://www.nvidia.com/en-us/ai-data-science/foundation-models/)交互！具体来说，您将使用 [**NVIDIA AI Foundation Models**](https://catalog.ngc.nvidia.com) 构建 AI 工作流原型，并编排一个相对复杂的自然语言应用。

$$---$$


几乎所有领域都面临部署大规模深度学习模型这项极具挑战性的任务。现在的模型，比如 Llama 2（70B 参数）或 Mixtral 7x8B 这种集成模型，是先进的训练方法、大量数据和强大计算系统的产物。幸运的是，这些模型已经过训练，可以用来实现许多场景了。然而，真正的挑战在于高效地托管这些模型。

**大模型的部署场景：**

1. **在高端数据中心部署：**
> 将未经压缩、量化的模型部署在配备高端 NVIDIA 显卡的数据中心上，实现快速推理和实验。
> * **优点：**该方案是可扩展部署和实验的理想选择，非常适合执行大型训练工作流或同时支持多个用户或模型。
> * **缺点：**为使用服务的每个用户分配资源的效率很低，除非是用来训练/微调模型或需要与底层模型组件交互。

2. **在有限的数据中心/专业消费级硬件部署：**
> 经过量化和优化的模型可以在一般的 GPU 数据中心甚至是先进的消费级显卡上运行。
> * **优点：**该方案能在单用户应用的性能和响应速度间取得平衡。这些会话也可以按用户部署，一次运行一到两个大型模型，并能访问模型内部（尽管这些模型需要经过量化）。
> * **缺点：**为每个用户部署实例的成本依然很高，当然对于一些简单的工作来说还挺合适的。但让用户在本地环境访问这些资源可能是不合理的。

3. **消费级硬件部署：**
> 尽管神经网络的能力会大幅受限，但大多数消费级硬件都具有图形界面（GUI）、可访问互联网的 Web 浏览器、一定量的内存（怎么也有 1 GB）和相当强大的 CPU。
> * **缺点：**目前大多数硬件都无法同时运行多个本地大型模型，哪怕只运行一个模型也需要大量的资源管理和优化。
> * **优点：**可以以此为基础考虑您的服务应支持哪些类型的用户。

您在本课程中使用的环境可以很好地代表典型的消费级硬件。虽然我们可以使用微服务，但依然受到仅 CPU 计算环境的限制，这样的环境将难以运行 LLM 模型。虽然这是一个巨大的限制，但我们仍然可以这样利用 LLM：

* 访问托管了大模型的高算力服务。
* 提供命令输入和展示结果的交互界面。

我们已经有了微服务和端口连接的基础，现在完全有能力探索有效的接口设计，让我们的开发环境能够访问 LLM！

----

<br>

## **第 2 部分：** 托管大模型服务

为了能在像我们这样的 CPU 环境中访问大语言模型（LLM），下面就来评估一下各种托管选项的可行性：

**“黑箱（Black-Box）”托管模型：**
> [OpenAI](https://openai.com/) 这种提供对 GPT-4 这种黑箱模型交互 API 的接口。这些功能强大且高度集成的服务可以为复杂的流程提供简单的接口，能够自动跟踪内存、调用其它模型，并根据使用场景整合多模态接口。同时，它们保持一定的透明性，通常不能简单的自行托管。
> * **优点：**开箱即用，对普通用户的门槛较低。
> * **缺点：**黑箱部署存在潜在的隐私问题，且面临定制程度受限、大规模应用成本高等问题。

**自托管模型：**

> 在几乎所有可扩展模型部署的背后，都有一个或多个在数据中心运行的巨型模型，它们拥有可扩展的资源和光速带宽。虽然有必要大规模部署大模型并控制开放接口，但这样的系统需要极强的专业背景才能搭建好，并且对非开发者的个人支持有限。这类系统更适合支持多用户、多个模型和自定义接口。
> * **优点：**它们提供集成自定义数据集和 API 的能力，为同时支持多用户而专门设计。
> * **缺点：**这种系统需要非常专业的技术背景才能正确的配置出来。

为了两全其美，我们将使用 [**NVIDIA NGC Service**](https://www.nvidia.com/en-us/gpu-cloud/)。NGC 提供了一套用于设计和部署 AI 解决方案的开发者工具。我们最需要的是 [NVIDIA AI Foundation Models](https://www.nvidia.com/en-us/ai-data-science/foundation-models/)，这里的模型是预先调整和优化好的，旨在方便开箱即用的可扩展部署（可以按原样部署，也可以进行一些自定义）。此外，NGC 可以在[可扩展的 DGX 加速计算环境](https://www.nvidia.com/en-us/data-center/dgx-platform/)下托管模型入口。

----

<br>

## **第 3 部分：** 托管推理服务

**部署可扩展的推理模型时，您通常需要执行以下步骤：**

* 确定您希望用户访问的模型，并分配托管这些模型的资源。
* 确定用户掌控模型的程度，并将访问方式暴露出来。
* 创建监控系统来跟踪/控制接口的访问，根据需要设置扩展和节流系统。

在本课程中，您将使用 NVIDIA 部署的模型，这些模型被托管为 **LLM NIM**。NIMs 是针对可扩展的 AI 推理服务优化过的微服务。它们非常适合本地推理，还提供了标准化的 API，但主要还是针对可扩展环境设计的。这些特定模型是作为共享函数部署在 NVIDIA DGX Cloud 上的，使用类似 OpenAPI 的 API 设计风格。我们来逐步分解这意味着什么：

**在集群端：**这些微服务托管在 Kubernetes 平台上，可以根据 DGX 节点的数量自由扩展，并以单个功能的形式提供服务。换言之：

* 将大语言模型下载到 **GPU 支持的计算节点**。
* 在启动时，系统会启动一部分计算节点，使得每个用户请求都会到达其中的某个节点。
	+ Kubernetes 将合理地路由此流量。如果有空闲的计算节点，那流量就会直接给它。如果所有节点都在工作，那么请求将排队等待最快被释放的节点。
	+ 在我们的环境中，由于启用了动态批处理（in-flight batching）功能，这些节点仍将非常快速地接收请求。这意味着每个节点在完全“被占满”之前，最多能接收 256 个请求。（256 是部署时设置的超参数）
* 随着负载增加，系统将启动自动扩展来尽可能避免延迟响应。

下图显示了使用自定义（非 OpenAPI）API 实现的一个函数调用。这是公共入口的默认调用方式，这个图展示了实现细节。

> <img src="https://dli-lms.s3.amazonaws.com/assets/s-fx-15-v1/imgs/ai-playground-api.png" width=800px/>

**在网关端：**为了使此 API 更标准化，API 网关服务器便将这些功能聚合到名为 OpenAPI 的通用 API 中。包括 OpenAI 在内的许多公司都采用了此规范，因此我们可以放心地使用 OpenAI：

> <img src="https://dli-lms.s3.amazonaws.com/assets/s-fx-15-v1/imgs/mixtral_api.png" width=800px/>

在本课程中，您会用一个更专用的接口，以连接到一个叫作 LangChain 的 LLM 编排框架（稍后会详细介绍）。您将使用更定制的接口，比如 [`langchain_nvidia_ai_endpoints`](https://python.langchain.com/docs/integrations/chat/nvidia_ai_endpoints/) 的 `ChatNVIDIA` 库。稍后将详细介绍。

**在用户端：**通过将这些入口整合到客户端中，您可以设计集成方式、工作流和用户体验，利用这些生成式 AI 功能为您的应用赋予推理和生成能力。着类应用的一个热门示例是 [OpenAI 的 ChatGPT](https://chat.openai.com/)，这是通过对入口（包括 GPT4、DALLE 等）进行编排实现的。虽然有时看起来像是单独的一个智能模型，但它实际上是模型入口的聚合，并采用工程化的方式帮助管理状态和上下文。整个课程都将强化您对此的理解，课程最后您就会知道怎么创建任意应用场景下的聊天助手了。

> <img src="https://dli-lms.s3.amazonaws.com/assets/s-fx-15-v1/imgs/openai_chat.png" width=700px/>

----

<br>

## **第 4 部分：[练习]** 尝试 Foundation Model 入口

从本节往后，您将开始使用入口！

**在您自己的（本地）环境中：**您可以打开 [`build.nvidia.com`](https://build.nvidia.com/) 找找您想用的模型。比如，您可以看看 [MistralAI 的 Mixtral-8x7b 模型](https://build.nvidia.com/mistralai/mixtral-8x7b-instruct)。您会在那里看到模型的用例、进一步了解的阅读链接，以及“Apply To Self-Host”（自行托管）和“Get API Key”(获取 API Key) 等按钮。

* 单击**“Apply To Self-Host”**将引导您了解 NVIDIA 微服务的信息，并为您提供注册途径（即抢先体验，或者通过 NVIDIA AI Enterprise 获得权限），或者加入通知列表（通用的访问途径）。
* 单击**“Get API Key”**将生成以“nvapi-”开头的 API 密钥，这样就可以通过网络请求来访问 API 入口了！

如果您想试试，可以将 API 密钥像这样加到 notebook 环境中。

In [ ]:
# import os
# os.environ["NVIDIA_API_KEY"] = "nvapi-..."

**在您的课程环境：**为了完成课程，我们将直接使用这些模型，并通过在本课程 `llm_client` 目录（即 [`llm_client/client_server.py`](llm_client/client_server.py)）中设置的代理服务器来访问模型。实现详情不在课程的讨论范围内，但您可以通过以下方式无限制地访问一部分模型：

* 暴露一些能将请求传给选定模型的入口。
* 在 llm_client 微服务中填写 API 密钥，以防止用完您的 credits。

**同样的代码也可以用来实现您自己的 GenAI 网关服务，就像 [`integrate.api.nvidia.com`](https://docs.api.nvidia.com/nim/reference/nvidia-embedding-2b-infer) 或 [`api.openai.com`](https://platform.openai.com/docs/api-reference/introduction) 。**

### **4.1.** 手动 Python 请求

正如我们之前说过的，您可以使用 Python 的 `requests` 库与微服务或远程 API 交互，通常会遵循以下流程：
* **导入库：**我们先要导入支持 HTTP 请求的 requests 库，以及处理 JSON 数据的 json 库。
* **API URL 和标头（headers）：**定义 API 入口的 URL 和标头，包括授权 （API key）和数据格式。
* **数据负载：**指定您要发送的数据，这里即一个简单的问句（query）。
* **发出请求：**用 `requests.post` 发送 POST 请求。您可以将 `post` 替换为 `get`，`put` 等等，取决于 API 的具体要求。
* **响应处理：**检查状态代码来确定请求是否成功（200 表示成功），然后再处理返回的数据。

下面我们来看看它具体都能提供什么样的入口和模型：

In [ ]:
import requests

invoke_url = "http://llm_client:9000"
headers = {"content-type": "application/json"}

requests.get(invoke_url, headers=headers, stream=False).json()

In [ ]:
import requests

invoke_url = "http://llm_client:9000/v1/models"
# invoke_url = "https://api.openai.com/v1/models"
# invoke_url = "https://integrate.api.nvidia.com/v1"
# invoke_url = "http://llm_client:9000/v1/models/mistralai/mixtral-8x7b-instruct-v0.1"
# invoke_url = "http://llm_client:9000/v1/models/mistralaimixtral-8x7b-instruct-v0.1"
headers = {
    "content-type": "application/json",
    # "Authorization": f"Bearer {os.environ.get('NVIDIA_API_KEY')}",
    # "Authorization": f"Bearer {os.environ.get('OPENAI_API_KEY')}",
}

print("Available Models:")
response = requests.get(invoke_url, headers=headers, stream=False)
# print(response.json())  ## <- Raw Response. Very Verbose
for model_entry in response.json().get("data", []):
    print(" -", model_entry.get("id"))

print("\nExample Entry:")
invoke_url = "http://llm_client:9000/v1/models/mistralai/mixtral-8x7b-instruct-v0.1"
requests.get(invoke_url, headers=headers, stream=False).json()

我们不会在本课程中过多的触及这种级别的抽象，但值得通过这个基本流程来确认这些请求是通过我们的微服务发出的，就好像是从远程托管服务器发送的一样。本课程之后的内容里，都可以假设您的客户端在背后进行这样的交互：

In [ ]:
from getpass import getpass
import os

## Where are you sending your requests?
invoke_url = "http://llm_client:9000/v1/chat/completions"

## If you wanted to use your own API Key, it's very similar
# if not os.environ.get("NVIDIA_API_KEY", "").startswith("nvapi-"):
#     os.environ["NVIDIA_API_KEY"] = getpass("NVIDIA_API_KEY: ")
# invoke_url = "https://integrate.api.nvidia.com/v1/chat/completions"

## If you wanted to use OpenAI, it's very similar
# if not os.environ.get("OPENAI_API_KEY", "").startswith("sk-"):
#     os.environ["OPENAI_API_KEY"] = getpass("OPENAI_API_KEY: ")
# invoke_url = "https://api.openai.com/v1/models"

## Meta communication-level info about who you are, what you want, etc.
headers = {
    "accept": "text/event-stream",
    "content-type": "application/json",
    # "Authorization": f"Bearer {os.environ.get('NVIDIA_API_KEY')}",
    # "Authorization": f"Bearer {os.environ.get('OPENAI_API_KEY')}",
}

## Arguments to your server function
payload = {
    "model": "mistralai/mixtral-8x7b-instruct-v0.1",
    "messages": [{"role":"user","content":"Tell me hello in French"}],
    "temperature": 0.5,   
    "top_p": 1,
    "max_tokens": 1024,
    "stream": True                
}

In [ ]:
import requests
import json

## Use requests.post to send the header (streaming meta-info) the payload to the endpoint
## Make sure streaming is enabled, and expect the response to have an iter_lines response.
response = requests.post(invoke_url, headers=headers, json=payload, stream=True)

## If your response is an error message, this will raise an exception in Python
try: 
    response.raise_for_status()  ## If not 200 or similar, this will raise an exception
except Exception as e:
    # print(response.json())
    print(response.json())
    raise e

## Custom utility to make live a bit easier
def get_stream_token(entry: bytes):
    """Utility: Coerces out ['choices'][0]['delta'][content] from the bytestream"""
    if not entry: return ""
    entry = entry.decode('utf-8')
    if entry.startswith('data: '):
        try: entry = json.loads(entry[5:])
        except ValueError: return ""
    return entry.get('choices', [{}])[0].get('delta', {}).get('content')

## If the post request is honored, you should be able to iterate over 
for line in response.iter_lines():
    
    ## Without Processing: data: {"id":"...", ... "choices":[{"index":0,"delta":{"role":"assistant","content":""}...}...
    # if line: print(line.decode("utf-8"))

    ## With Processing: An actual stream of tokens printed one-after-the-other as they come in
    print(get_stream_token(line), end="")

<br>

#### **[注意]**

**您可能注意到聊天模型需要输入“messages”：**

如果您已经熟悉 HuggingFace 上模型的原始 LLM 接口，可能会觉得惊讶，但对于用过 OpenAI 模型的用户来说这是很自然的。使用受限的接口而不是原始的补全文本接口，会让服务更好地限定用户的行为。这种接口有很多优缺点：
* 这种服务可以对角色（role）和参数进行限制（比如系统消息的限制、进行任意生成的启动消息等）。
* 服务可能想限定自定义提示的格式，并在背后实现聊天界面提供的其它选项。
* 您可能想让服务通过更强的假设来在推理流水线中实现更深层的优化。
* 服务可能会通过模仿另一个热门的接口来兼容现有生态。

所有这些理由都是合理的，因此在选择和部署您自己的服务时，请务必考虑哪种接口选项最符合您的场景。

**您可能会注意到两种调用模型的方式：**

您可以**不使用流式传输**来调用模型，这样服务响应会等全部计算完成之后再立刻返回。当您在执行其它操作前需要模型的完整输出时，这非常有用，比如您想打印整个结果或将其用于下游任务时。响应体如下所示：

```json
{
    "id": "d34d436a-c28b-4451-aa9c-02eed2141ed3",
    "choices": [{
        "index": 0,
        "message": { "role": "assistant", "content": "Bonjour! ..." },
        "finish_reason": "stop"
    }],
    "usage": {
        "completion_tokens": 450,
        "prompt_tokens": 152,
        "total_tokens": 602
    }
}
```

您还可以用**流式传输**的方式调用，这时服务将发送一系列请求，直到发完最后一个。当您想在服务响应可用时就拿到时会非常有用（这对于想在生成的过程中同时向用户打印输出的语言模型组件非常有用）。在这种情况下，响应体看起来会是这样：

```json
data:{"id":"...","choices":[{"index":0,"delta":{"role":"assistant","content":"Bon"},"finish_reason":null}]}
data:{"id":"...","choices":[{"index":0,"delta":{"role":"assistant","content":"j"},"finish_reason":null}]}
...
data:{"id":"...","choices":[{"index":0,"delta":{"role":"assistant","content":""},"finish_reason":"stop"}]}
data:[DONE]
```

这两中方式都可以通过 Python 的 `requests` 库来实现，但会导致大量重复代码。幸运的是，我们有一些更易用，且能整合到大型项目中的系统！

### **4.2.** OpenAI 客户端请求

我们需要知道有这么个接口存在，但原样使用会导致大量重复代码并引入额外的复杂性。幸运的是，我们有一些更易用，且能整合到大型项目中的系统！可以通过像 OpenAI 这样经过设计的客户端来从更高的抽象发出请求。由于 NVIDIA 和 OpenAI 都遵循着 OpenAPI 的规范，我们可以直接借用 OpenAI 的客户端。请注意，同样的流程依然在背后执行，可能是由较低级别的客户端（比如 [httpx](https://github.com/projectdiscovery/httpx) 或 [aiohttp](https://github.com/aio-libs/aiohttp) 来完成的。

In [ ]:
## Using General OpenAI Client
from openai import OpenAI

# client = OpenAI()  ## Assumes OPENAI_API_KEY is set

# client = OpenAI(
#     base_url = "https://integrate.api.nvidia.com/v1",
#     api_key = os.environ.get("NVIDIA_API_KEY", "")
# )

client = OpenAI(
    base_url = "http://llm_client:9000/v1",
    api_key = "I don't have one"
)

completion = client.chat.completions.create(
    model="mistralai/mixtral-8x7b-instruct-v0.1",
    # model="gpt-4-turbo-2024-04-09",
    messages=[{"role":"user","content":"Hello World"}],
    temperature=1,
    top_p=1,
    max_tokens=1024,
    stream=True,
)

## Streaming with Generator: Results come out as they're generated
for chunk in completion:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")

In [ ]:
## Non-Streaming: Results come from server when they're all ready
completion = client.chat.completions.create(
    model="mistralai/mixtral-8x7b-instruct-v0.1",
    # model="gpt-4-turbo-2024-04-09",
    messages=[{"role":"user","content":"Hello World"}],
    temperature=1,
    top_p=1,
    max_tokens=1024,
    stream=False,
)

completion

### **4.3.** ChatNVIDIA 客户端请求

到目前为止，我们已经看到了在两个层次的抽象层上进行的通信：**原始请求**和 **API 客户端**。在本课程中，我们希望使用名为 LangChain 的框架进行 LLM 编排，因此我们需要进行更高的一层的抽象，**框架连接器**（Framework Connector）。

**连接器**的目标是将任意 API 从其原生的核心转换为目标代码库所期望的 API。在本课程中，我们希望利用 LangChain 的以链为中心的生态，但原始的 `requests` API 帮不了我们。在 LangChain 的背后，每个非本地托管的聊天模型都必须依赖此类 API，但面向开发者的 API 是更简洁的 [`LLM` 或 `SimpleChatModel` 风格的接口](https://python.langchain.com/docs/modules/model_io/)，它们有默认的参数和一些简单实用的函数，比如 `invoke` 和 `stream`。

要开始探索 LangChain，我们将使用 `ChatNVIDIA` 连接器与 `chat/completions` 入口进行交互。该模型是 LangChain 生态系统的一部分，可以通过 `pip install langchain-nvidia-ai-endpoints` 在本地安装。

In [ ]:
## Using ChatNVIDIA
from langchain_nvidia_ai_endpoints import ChatNVIDIA

## NVIDIA_API_KEY pulled from environment
llm = ChatNVIDIA(model="mistralai/mixtral-8x7b-instruct-v0.1")
# llm = ChatNVIDIA(model="mistralai/mixtral-8x7b-instruct-v0.1", mode="open", base_url="http://llm_client:9000/v1")
llm.invoke("Hello World")

In [ ]:
llm._client.last_inputs

In [ ]:
# llm._client.last_response
llm._client.last_response.json()

<br/>

#### **[注意]**

**本课程使用的是一个修改过的 `ai-endpoints` 连接器分支，它包括一些对我们的课程环境更有帮助的特性。**这些特性还不在 main 中，但正在积极的从 [**LlamaIndex**](https://docs.llamaindex.ai/en/stable/examples/embeddings/nvidia/) 和 [**Haystack**](https://docs.haystack.deepset.ai/docs/nvidiagenerator) 中合并。

**ChatNVIDIA 会默认使用 `llm_client` 微服务，是因为我们是像这样设置环境变量的：**

In [ ]:
import os

{k:v for k,v in os.environ.items() if k.startswith("NVIDIA_")}
## Translation: Use the base_url of llm_client:9000 for the requests,
## and use "open"api-spec access for model discovery and url formats

**在整个课程中，您都可以随时尝试您选择的模型。**下面是本课程中提供的所有模型。

您也可以尝试其它模型，但可能需要自行升级 `langchain_nvidia_ai_endpoints` 库并使用自己的 key。

In [ ]:
model_list = ChatNVIDIA.get_available_models(list_none=False)

for model_card in model_list:
    model_name = model_card.id
    llm = ChatNVIDIA(model=model_name)
    print(f"TRIAL: {model_name}")
    try: 
        for token in llm.stream("Tell me about yourself! 2 sentences.", max_tokens=100):
            print(token.content, end="")
    except Exception as e: 
        print(f"EXCEPTION: {e}")
    print("\n\n" + "="*84)

----

<br>

## **第 5 部分：** 总结

此 notebook 的目的是围绕 LLM 服务托管策略进行一些讨论，并向您介绍 AI Foundation Model 入口。我们希望您现在能够直观地了解边缘设备如何提供或访问远程 LLM 服务！

### <font color="#76b900">**非常好！**</font>

### 接下来：
**[可选]** 回顾 notebook 顶部的“思考问题”。